In [241]:
import pandas as pd
from dotenv import load_dotenv
from googlemaps import convert
import googlemaps
import os
import requests
import json
import foursquare
from config import dic_categorias
from pymongo import MongoClient
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import matplotlib 
from selenium import webdriver


In [2]:
load_dotenv()

True

In [3]:
tokg = os.getenv("tokg")
tokf3 = os.getenv ("tokf3")
client2= MongoClient ("localhost:27017")

In [5]:
client = foursquare.Foursquare( access_token= tokf3 )
gmaps = googlemaps.Client(key= tokg )
db = client2.get_database("ciudades")

In [185]:
dic_categorias= {
#infraestructuras transporte
    "aeropuertos" : ["airport"],
    "estaciones de tren": ["train station", "estacion de tren"],  
    "parques": ["park"],
    "estaciones de metro": ["metro", "subway station"],
    "estaciones de autobús": ["bus"],
    "carril bici": ["bike trail"],
# infraestructuras sanidad
    "hospitales" : ["hospital"],
    "clinicas": ["medical center", "clinic"],
    "farmacias": ["pharmacy", "farmacia"],
    "veterinario": ["veterinarian"],
#restauracion
    "restaurantes": ["restaurant"],
    "ocio_nocturno": ["pub","Cocktail Bar"],
    "hoteles": ["hotel"],
    "cafeterias": ["coffe"],
#ocio y cultura
    "cines": ["cinema", "movie theatre"],
    "teatros": ["theatre", "teatro"],
    "museos": ["museum"],
    "salas_de_conciertos": ["rock club", "concert"],
    "bibliotecas" : ["library", "biblioteca"], 
#ocio y deporte
    "ocio_deporte": ["gym", "Fitness"],
    "piscina": ["pool", "piscina"],
    "canchas": ["cancha", "court"],
    "estadios" : ["stadium", "estadio"],
    
#infraestructuras educacion
    "colegios" : ["primary school", "elemental school"],
    "guarderias" : ["kindergarden", "guarderia"],
    "universidades": ["college", "university"],
    "Institutos" : ["secundary school", "bachiller"],
    "Escuelas" : ["school", "escuela"],
    
#comercio
    "centros_comerciales": ["mall", "shopping center"],
    "Tiendas minoristas" : ["shop", "tienda"]
    
                             }

In [172]:
""""
dic_categorias= {
#infraestructuras transporte
   
    "estaciones de tren": ["train station", "estacion de tren"],  
   
# infraestructuras sanidad
    
    "farmacias": ["pharmacy", "farmacia"],
    
#restauracion
    "restaurantes": ["restaurant"],

#ocio y cultura
    "cines": ["cinema", "movie theatre"],

#ocio y deporte
    "ocio_deporte": ["gym", "Fitness"],

    
#infraestructuras educacion

    "Escuelas" : ["school", "escuela"],
    
#comercio
    
    "Tiendas minoristas" : ["shop", "tienda"]
    
                             }
                             """

In [67]:
coord= "40.3829439,-3.7332451"

In [82]:
request = client.venues.search(params = {'query': ["kindergarden", "guarderia"] , 'll': coord, "radius": 1500, "limit": 50})

In [93]:
keys= list (dic_categorias.keys())

In [115]:
cat1= keys[:6]
cat2= keys[6:10]
cat3= keys[10:14]
cat4= keys [14:19]
cat5= keys [19:23]
cat6= keys [23:28]
cat7= keys[-2:]

In [6]:
def test (nombre):
    lista= db.list_collection_names()
    return nombre in lista       

In [280]:
coords2=gmaps.geocode( address= "campos eliseos, paris")


In [292]:
df["codigo postal"][1]

'02002'

In [298]:
len (df)

598

In [299]:
ciudad= "clara campoamor"
pais= "madrid"

In [300]:
df.columns

Index(['codigo postal', 'renta bruta media', 'renta disponible media'], dtype='object')

In [303]:
busqueda= ciudad + ", " + pais #unimos en un string para pasarselo como búsqueda a gplaces
coords=gmaps.geocode( address= busqueda)
nombre = coords[0]["address_components"][5]["short_name"] # nos quedamos con el nombre, que será el de la colección
a=coords[0]["geometry"]["location"]["lat"] 
b=coords[0]["geometry"]["location"]["lng"]
c = str (a) +"," + str(b) # Formato que luego entienda foursquare
for n in range (len (df)):
    if df["codigo postal"][n] == nombre:
        renta_neta= df["renta disponible media"][n]
        renta_bruta= df["renta bruta media"][n]
    else:
        pass
dic= {"nombre": nombre,
                "coordenadas": c,
                 "latitud":a,
                 "longitud":b,
                "renta bruta": renta_bruta,
                 "renta neta": renta_neta}

print (dic)

{'nombre': '28025', 'coordenadas': '40.3829439,-3.7332451', 'latitud': 40.3829439, 'longitud': -3.7332451, 'renta bruta': '24.346', 'renta neta': '20.457'}


In [8]:
def get_coords(nombre, dic):
    collection= nombre 
    db[collection].insert_one (dic)
    coord= dic["coordenadas"]
    return get_data_fields (coord, collection)
        

In [9]:
def get_data_fields (coord, collection):
    tokf3 = os.getenv ("tokf3")
    client = foursquare.Foursquare( access_token= tokf3 )
    cantidad=[]
    dic_insert={}
    for categoria, codigo in dic_categorias.items():  
        request = client.venues.search(params = {'query': codigo , 'll': coord, "radius": 1500, "limit": 50})
        lista=request["venues"]
        cantidad= len (lista)
        dice1={categoria: {"número":cantidad }}
        dice=[]
        for i in range (0,len (lista)): # para cada venue, sacame:
            nombre = lista[i]["name"]
            latitud= lista[i]["location"]["lat"]
            longitud= lista[i]["location"]["lng"]
            try:
                tipo = lista[i]["categories"][0]["name"]
            except:
                pass
            try:
                direccion= lista[i]["location"]["formattedAddress"]
            except:
                pass
        
            dice2 = {"nombre": nombre, 
                       "latitud":latitud,
                       "longitud": longitud,
                        "tipo" : tipo,
                        "direccion": direccion }
            dice.append (dice2)          
        
            dice1[categoria].update ({"listado": dice})
        db[collection].insert_one (dice1)
    
    return "se han introducido los campos"

In [186]:
def get_data_fields (coord, collection):

    for categoria, codigo in dic_categorias.items():  
        request = client.venues.search(params = {'query': codigo , 'll': coord, "radius": 1500, "limit": 50})
        lista=request["venues"]
        
        
        dice=[]
        for i in range (0,len (lista)): # para cada venue, sacame:
            nombre = lista[i]["name"]
            latitud= lista[i]["location"]["lat"]
            longitud= lista[i]["location"]["lng"]
            try:
                tipo = lista[i]["categories"][0]["name"]
            except:
                tipo= "no definido"
            try:
                direccion= str (lista[i]["location"]["formattedAddress"])[2:-2]
            except:
                direccion= "no disponible"
            
            dice2 = {"nombre": nombre,
                     "categoria": get_primary (categoria),
                "subcategoria": categoria, 
               "latitud":latitud,
               "longitud": longitud,
                "tipo" : tipo,
               "direccion":direccion }
            dice.append (dice2)          
        
        try:   # cuando dice está vacio, dá error, así lo obviamos.
            db[collection].insert_many (dice)
        except:
            pass
    
    return (collection)

In [390]:
def query(collection, items, numero):
    # ver si se pueden poner varios items, modificar funcion caso
    a=list (db[collection]. find ({item: {"$exists": True}},{ item +".listado" :1, "_id":0}))
    try:
        listapeque= a[0]["restaurantes"]["listado"][:numero]
    except:
        return "No hay resultados que mostrar"
        
    df=pd.DataFrame (listapeque)
    
    return making_map (df,collection)

In [187]:
collection= test_and_create ("calle jilguero", "madrid")

In [506]:
db[collection].find_one ()

{'_id': ObjectId('60a44d02bc723d4dbad85c8f'),
 'nombre': 'Av. Oporto, Madrid, Spain',
 'coordenadas': '40.3856358,-3.7250457',
 'latitud': 40.3856358,
 'longitud': -3.7250457}

In [27]:
def get_map (collection, items):
    lista=[]
    for item in items:
        a=list (db[collection]. find ({"categoria": item},{"latitud":1,"nombre":1, "longitud":1, "categoria":1, "_id":0}).limit(15))
        lista.append (a)
    cosa=[]
    for n in range (len(lista)):
        cosa+= lista[n]
    if len (cosa)==0:
        return False
    else:
        df=pd.DataFrame (cosa)
    
    return making_map (df, collection)

In [22]:
def making_map (df,collection):
    
    a=list (db[collection]. find ({"coordenadas": {"$exists": True}},{"latitud":1,"longitud" :1, "nombre":1,"_id":0})) 
    
    lat=a[0]["latitud"]
    
    long=a[0]["longitud"]
    
    nombre= a[0]["nombre"]
    
    mapa = folium.Map(location= [lat, long], zoom_start= 14, tooltip=nombre)
    for i, row in df.iterrows():
        
        marcador = {
            "location":[row["latitud"], row["longitud"]],
            "tooltip" : row["categoria"]
        }
    
        if row["categoria"] == "Festivo":
            icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
        )
        
        elif row["categoria"] == "Vispera":
            icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "home",
                    icon_color = "black"
        )
        
        else:
            icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "briefcase",
                    icon_color = "black"
            )
        Marker(**marcador,icon = icon ).add_to(mapa)
    return mapa
    

In [61]:
def grafica_post(collection):
    dic_categorias= {
    
    "restaurantes": ["restaurant"],
    "playas": "4bf58dd8d48988d1e2941735",
    "ocio_nocturno": ["pub","Cocktail Bar"],
    "hoteles": ["hotel"],
    "ocio_deporte": ["gym", "Fitness"],
    "colegios" : ["school", "primary school", "elemental school"],
    "guarderias" : ["kindergarden", "preschool"],
    "universidades": ["college", "university"],
    "aeropuertos" : ["airport"],
    "estaciones de tren": ["train station"],
    "parques": ["park"],
    "hospitales" : ["hospital", "medical center"],
    "cines": ["cinema", "movie theatre"],
    "teatros": ["theatre"],
    "museos": ["museum"],
    "salas_de_conciertos": ["rock club", "jazz club", "concert"],
    "piscina_para_Emilio": ["pool"],
    "bibliotecas" : ["library", "book store"],               
    "centros_comerciales": ["mall", "shopping center"]
    
                                            }
    c= list (dic_categorias.keys())
    lista=[]
    for item in c:
        a=list (db[collection].find ({"categoria": item},{ item :1, "_id":0}))
        cantidad= len (a)
        lista.append ( { collection : cantidad } )
    df=pd.DataFrame (lista, index=c)
    return df

In [182]:
def get_primary (categoria):
    keys= list (dic_categorias.keys())
    cat1= keys[:6]
    cat2= keys[6:10]
    cat3= keys[10:14]
    cat4= keys [14:19]
    cat5= keys [19:23]
    cat6= keys [23:28]
    cat7= keys[-2:]
    
    if categoria in cat1:
        clase = "Infraestructura transporte"
        return clase
    elif categoria in cat2:
        clase= "Infraestructura sanidad"
        return clase
    elif categoria in cat3:
        clase= "Ocio y Restauración"
        return clase
    elif categoria in cat4:
        clase= "Ocio y cultura"
        return clase
    elif categoria in cat5:
        clase= "Ocio y deporte"
        return clase
    elif categoria in cat6:
        clase= "Infraestructuras educacion"
        return clase
    elif categoria in cat7:
        clase= "Comercio"
        return clase
    else:
        return "desconocido"
   
    

In [183]:
get_primary ("restaurantes")

'Infraestructura transporte'

In [179]:
clase

'Comercio'

In [141]:
cantidad=[]
dic_insert={}
for categoria, codigo in dic_categorias.items():  
    request 
    lista=request["venues"]
    cantidad= len (lista)
    dice1={categoria: {"número":cantidad }}
    dice=[]
    print (categoria)  
    for i in range (0,len (lista)): # para cada venue, sacame:
        nombre = lista[i]["name"]
        latitud= lista[i]["location"]["lat"]
        longitud= lista[i]["location"]["lng"]

        dice2 = {"nombre": nombre, 
                       "latitud":latitud,
                       "longitud": longitud,
                        "subcategoria": categoria,
                        "categoria": clase}
        dice.append (dice2)          
        print (dice2)
        dice1[categoria].update ({"listado": dice})
    db[collection].insert_one (dice1)
    


aeropuertos
{'nombre': 'Guardería San Roque', 'latitud': 40.38952195045214, 'longitud': -3.736509681444763, 'subcategoria': 'aeropuertos', 'categoria': 'Comercio'}
{'nombre': 'En La Guarderia Las Amapolas', 'latitud': 40.370525, 'longitud': -3.741557, 'subcategoria': 'aeropuertos', 'categoria': 'Comercio'}
{'nombre': 'Guardería Nido La Cigüeña', 'latitud': 40.38136968264962, 'longitud': -3.7490762782583356, 'subcategoria': 'aeropuertos', 'categoria': 'Comercio'}
estaciones de tren
{'nombre': 'Guardería San Roque', 'latitud': 40.38952195045214, 'longitud': -3.736509681444763, 'subcategoria': 'estaciones de tren', 'categoria': 'Comercio'}
{'nombre': 'En La Guarderia Las Amapolas', 'latitud': 40.370525, 'longitud': -3.741557, 'subcategoria': 'estaciones de tren', 'categoria': 'Comercio'}
{'nombre': 'Guardería Nido La Cigüeña', 'latitud': 40.38136968264962, 'longitud': -3.7490762782583356, 'subcategoria': 'estaciones de tren', 'categoria': 'Comercio'}
parques
{'nombre': 'Guardería San Roqu

In [133]:
keys= list (dic_categorias.keys())

In [134]:
cat1= keys[:6]
cat2= keys[6:10]
cat3= keys[10:14]
cat4= keys [14:19]
cat5= keys [19:23]
cat6= keys [23:28]
cat7= keys[-2:]

In [212]:
collection

'Calle Jilguero, 28025 Madrid, Spain'

In [215]:
a= list (db["Calle Jilguero, 28025 Madrid, Spain"].distinct("subcategoria"))

In [219]:
collection=test_and_create ("calle mayor", "madrid")


In [190]:
def grafica_cat(collection):
    """
    Clon del anterior, pero con una coleción definida
    """
  
    lista=[]
    for item in lista_items():
        a=list (db[collection].find ({"categoria": item},{ item :1, "_id":0}))
        cantidad= len (a)
        lista.append ( { collection : cantidad } )
    df=pd.DataFrame (lista)
    return df

In [191]:
grafica_cat (collection)

NameError: name 'lista_items' is not defined

In [218]:
def grafica_intro():
    """
    función que crea un dataframe con la cantidad de documentos de cada categoria para una colección aleatoria realizando querys para cada
    categoría y quedándose con el len
    """
  
    collection= random.choice (db.list_collection_names())
    lista=[]
    for item in list_categories(collection):
        a=list (db[collection].find ({"categoria": item},{ item :1, "_id":0}))
        cantidad= len (a)
        lista.append ( { collection : cantidad } )
    df=pd.DataFrame (lista, index=list_categories(collection))
    return df

In [223]:
grafica_intro ()

""


In [225]:
def list_categories (collection):
    return (list (db[collection].distinct("categorias")))

SyntaxError: invalid syntax (<ipython-input-225-198649ed0cfe>, line 2)

In [ ]:
list (db[collection].find ({"categoria": item},{ item :1, "_id":0}))

In [224]:
list_categories (collection)

[]

In [228]:
db['Calle Mayor, Madrid, Spain'].distinct("categoria")

[]

In [227]:
collection

'Calle Mayor, Madrid, Spain'

In [229]:
request

{'venues': [{'id': '50a50321e4b0acf883d480ab',
   'name': 'Guardería San Roque',
   'location': {'address': 'Calle De La Oca 35',
    'lat': 40.38952195045214,
    'lng': -3.736509681444763,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.38952195045214,
      'lng': -3.736509681444763}],
    'distance': 782,
    'cc': 'ES',
    'city': 'Madrid',
    'state': 'Madrid',
    'country': 'España',
    'formattedAddress': ['Calle De La Oca 35', 'Madrid Madrid']},
   'categories': [{'id': '4f4533814b9074f6e4fb0107',
     'name': 'Nursery School',
     'pluralName': 'Nursery Schools',
     'shortName': 'Nursery School',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1621396357',
   'hasPerk': False},
  {'id': '4fec8037e4b0d7135b7c37a7',
   'name': 'En La Guarderia Las Amapolas',
   'location': {'lat': 40.370525,
    'lng': -3.741557,
    'labeledLatLngs': [{'label': 'display',


In [235]:
a=gmaps.places_nearby(
    location= coord,
    radius=1500,
    keyword= "restaurante"
)

In [236]:
a.keys()

dict_keys(['html_attributions', 'next_page_token', 'results', 'status'])

In [239]:
a["results"][0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 40.3800005, 'lng': -3.7490893},
  'viewport': {'northeast': {'lat': 40.38160932989273,
    'lng': -3.747422920107277},
   'southwest': {'lat': 40.37890967010729, 'lng': -3.750122579892721}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/bar-71.png',
 'name': 'El Rincón del Greco',
 'opening_hours': {'open_now': False},
 'photos': [{'height': 3120,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112670451980288672610">A Google User</a>'],
   'photo_reference': 'ATtYBwL-9tBPqatfDV933qyyAq00HOA76Le0zrk4KI6ET0UiLV-XzZcAyJ5jY4NrVIFarvvglZgnpub17QC4ob1olZxp-bML5rx__Lb-e3WlIi1dGe6y7xRJqtzPHLQDqNWzwOoTJxRlRiL8EZSvmTbiXNY1SuPsbwT0mEDJbzr-zvPIcaUr',
   'width': 4160}],
 'place_id': 'ChIJdd2wTi-IQQ0R32yOhGKM3pk',
 'plus_code': {'compound_code': '97J2+29 Madrid',
  'global_code': '8CGR97J2+29'},
 'price_level': 1,
 'rating': 4.5,
 'reference': 'ChIJdd2wTi-IQQ0R32yOhGKM3pk',
 'scope

In [247]:
body=driver.find_element_by_xpath ("/html/body/div/div/section/div/div[4]/div/article/div[2]/div/div[3]/div/table/tbody")

In [249]:
tabladatos=[]
for dato in body.find_elements_by_tag_name("tr"):
    fila=[d for d in dato.find_elements_by_tag_name ("td")]
    if len (fila)>1 :
        fila= {
            "codigo postal": fila[0].text,
            "renta bruta media": fila[1].text,
            "renta disponible media": fila[2].text
             }
        tabladatos.append (fila)

In [261]:
df= pd.DataFrame (tabladatos)

In [262]:
df

,codigo postal,renta bruta media,renta disponible media
0,02001-San Antonio Abad- La Pajarita- Polígono ...,32.036,25.823
1,02002-Parque Sur-Huerta de Marzo (Albacete),30.426,24.615
2,02003-Franciscanos (Albacete),26.866,22.145
3,02004-Fátima- Villacerrada (Albacete),23.072,19.449
4,02005-Industria- El Pilar- Feria- Ntra. Sra. d...,25.098,20.822
...,...,...,...
593,50190-Garrapinillos (Zaragoza),29.913,24.229
594,50194-Movera-Torre Larraz (Zaragoza),27.976,22.941
595,50620-Casetas (Zaragoza),22.356,18.812
596,50720-Cartuja Baja (Zaragoza),26.286,21.398


In [255]:
a=df["codigo postal"][1]

In [257]:
def limpia_codigo (a):
    return a[:5]
    

In [256]:
a

'02002-Parque Sur-Huerta de Marzo (Albacete)'

In [258]:
limpia_codigo (a)

'02002'

In [263]:
df["codigo postal"]= df["codigo postal"].apply (limpia_codigo)

In [264]:
df

,codigo postal,renta bruta media,renta disponible media
0,02001,32.036,25.823
1,02002,30.426,24.615
2,02003,26.866,22.145
3,02004,23.072,19.449
4,02005,25.098,20.822
...,...,...,...
593,50190,29.913,24.229
594,50194,27.976,22.941
595,50620,22.356,18.812
596,50720,26.286,21.398


In [265]:
df.to_csv ("../Data/rentaporcp.csv")

In [306]:
rentacodigo = pd.read_csv ("../Data/rentaporcp.csv") 

In [317]:
rentacodigo

,Unnamed: 0,codigo postal,renta bruta media,renta disponible media
0,0,2001,32.036,25.823
1,1,2002,30.426,24.615
2,2,2003,26.866,22.145
3,3,2004,23.072,19.449
4,4,2005,25.098,20.822
...,...,...,...,...
593,593,50190,29.913,24.229
594,594,50194,27.976,22.941
595,595,50620,22.356,18.812
596,596,50720,26.286,21.398


In [361]:
def test_and_create( ubicacion, region):
    
    """ 
        Esta función recibo dos inputs del usuario, los utiliza para hacer una geolozalización mediante la api de google places, de la que
        obtengo sus coordenadas, que guardamos en 2 formatos, y la dirección, que usaremos como nombre para las colecciones. Meto las variables
        que me interesan en un diccionario, compruebo que la colección no exista por el nombre, si no existe la creo y cómo salida llamámos a
        otra función pasándole el diccionario y el nombre de la colección
    """
    try: 
        busqueda= ubicacion + ", " + region #unimos en un string para pasarselo como búsqueda a gplaces
        coords=gmaps.geocode( address= busqueda)
        nombre = coords[0]["address_components"][5]["short_name"]
        if len (nombre)<5:
            return "los datos introduucidos no son correctos"
        else:
            pass
        a=coords[0]["geometry"]["location"]["lat"] 
        b=coords[0]["geometry"]["location"]["lng"]
        c = str (a) +"," + str(b) # Formato que luego entienda foursquare
        print (nombre)
        for n in range (len (rentacodigo)):
            if rentacodigo["codigo postal"][n] == nombre: 
                renta_neta= rentacodigo["renta disponible media"][n]
                renta_bruta= rentacodigo["renta bruta media"][n]
    
                dic= {"nombre": nombre,
                "coordenadas": c,
                 "latitud":a,
                 "longitud":b,
                "renta bruta": renta_bruta,
                 "renta neta": renta_neta}
            else:
                pass
            
       
        
        else:
            
            pass
        
        db.create_collection (nombre)   
    except: "los datos introducidos no son correctos"
        
    return  dic

In [362]:
test_and_create ("28018", "madrid")

UnboundLocalError: local variable 'dic' referenced before assignment

In [311]:
def test (nombre):
    """
        Función que comprueba si la colleción existe, devuelve un booleano
    """
    lista= db.list_collection_names()
    return nombre in lista

In [468]:
coords=gmaps.geocode( address = "50820, madrid")

In [471]:
coords

[{'address_components': [{'long_name': 'Madrid',
    'short_name': 'Madrid',
    'types': ['locality', 'political']},
   {'long_name': 'Madrid',
    'short_name': 'M',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Community of Madrid',
    'short_name': 'MD',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Spain',
    'short_name': 'ES',
    'types': ['country', 'political']}],
  'formatted_address': 'Madrid, Spain',
  'geometry': {'bounds': {'northeast': {'lat': 40.5638447, 'lng': -3.5249115},
    'southwest': {'lat': 40.3120639, 'lng': -3.8341618}},
   'location': {'lat': 40.4167754, 'lng': -3.7037902},
   'location_type': 'APPROXIMATE',
   'viewport': {'northeast': {'lat': 40.5638447, 'lng': -3.5249115},
    'southwest': {'lat': 40.3120639, 'lng': -3.8341618}}},
  'place_id': 'ChIJgTwKgJcpQg0RaSKMYcHeNsQ',
  'types': ['locality', 'political']}]

In [484]:
import pgeocode
nomi = pgeocode.Nominatim("es")
a = nomi.query_postal_code("5151554")
a

postal_code       5151554
country_code          NaN
place_name            NaN
state_name            NaN
state_code            NaN
county_name           NaN
county_code           NaN
community_name        NaN
community_code        NaN
latitude              NaN
longitude             NaN
accuracy              NaN
Name: 0, dtype: object

In [497]:
rentacodigo= pd.read_csv ("../Data/rentaporcp.csv")

In [498]:
rentacodigo['codigo postal'] = rentacodigo['codigo postal'].apply(lambda x: "0" + str(x) if len(str(x))==4 else str(x))
#rentacodigo['codigo postal'] = rentacodigo['codigo postal'].apply(int)
rentacodigo['codigo postal']

0      02001
1      02002
2      02003
3      02004
4      02005
       ...  
593    50190
594    50194
595    50620
596    50720
597    50820
Name: codigo postal, Length: 598, dtype: object

In [505]:
def validar_codigo (cp):
    '''Esta función comprueba que e lcódigo postal devuevla una dirección y que tenga una longitud de 5'''
    
    coords=gmaps.geocode( address= cp + "," + pais)
    nombre = coords[0]["address_components"][0]["short_name"]
    
    if type(nombre) == int and len(nombre)==5:
        return nombre
    else:
        nomi = pgeocode.Nominatim("es")
        a = nomi.query_postal_code(cp)
        if a["latitude"]==np.nan:
            return "Latitude es un NaN"
        else:
            nombre=a["postal_code"]
            if len(nombre)==5:
                return nombre

In [508]:
def test_and_create(cp, pais): 
    """ 
        Esta función recibo dos inputs del usuario, los utiliza para hacer una geolozalización mediante la api de google places, de la que
        obtengo sus coordenadas, que guardamos en 2 formatos, y la dirección, que usaremos como nombre para las colecciones. Meto las variables
        que me interesan en un diccionario, compruebo que la colección no exista por el nombre, si no existe la creo y cómo salida llamámos a
        otra función pasándole el diccionario y el nombre de la colección
    """
    #Validamos el input
    name = validar_codigo(cp)
    
    #Comprobamos que no esté creada
    if test(nombre):
        return nombre
        
    #Definimos coordenadas
    a=coords[0]["geometry"]["location"]["lat"] 
    b=coords[0]["geometry"]["location"]["lng"]
    c = str (a) +"," + str(b) # Formato que luego entienda foursquare
    
    #Renta
    renta_neta=0
    renta_bruta=0
    
    for n in range (len(rentacodigo)):
        if rentacodigo["codigo postal"][n] == str(nombre):
                renta_neta += rentacodigo["renta disponible media"][n]
                renta_bruta += rentacodigo["renta bruta media"][n]
        else:
            print("El código postal no se encuentra en el df")
        
        #Creamos el diccionario
        dic= {          "nombre": nombre,
                        "coordenadas": c,
                        "latitud":a,
                        "longitud":b,
                        "renta bruta": renta_bruta,
                        "renta neta": renta_neta}
    
    #Creamos la colección
    db.create_collection (str(nombre))   
    
        
    return  get_ (nombre, dic)

In [509]:
import numpy as np
test_and_create ("50820", "madrid")

02001 50820
02002 50820
02003 50820
02004 50820
02005 50820
02006 50820
02008 50820
02049 50820
03001 50820
03002 50820
03003 50820
03004 50820
03005 50820
03006 50820
03007 50820
03008 50820
03009 50820
03010 50820
03011 50820
03012 50820
03013 50820
03014 50820
03015 50820
03016 50820
03112 50820
03113 50820
03195 50820
03201 50820
03202 50820
03203 50820
03204 50820
03205 50820
03206 50820
03208 50820
03291 50820
03292 50820
03293 50820
03294 50820
03295 50820
03296 50820
03320 50820
03540 50820
03699 50820
04001 50820
04002 50820
04003 50820
04004 50820
04005 50820
04006 50820
04007 50820
04008 50820
04009 50820
04120 50820
04130 50820
04131 50820
07001 50820
07002 50820
07003 50820
07004 50820
07005 50820
07006 50820
07007 50820
07008 50820
07009 50820
07010 50820
07011 50820
07012 50820
07013 50820
07014 50820
07015 50820
07120 50820
07198 50820
07199 50820
07600 50820
07610 50820
08001 50820
08002 50820
08003 50820
08004 50820
08005 50820
08006 50820
08007 50820
08008 50820
0800

{'nombre': 50820,
 'coordenadas': '40.4167754,-3.7037902',
 'latitud': 40.4167754,
 'longitud': -3.7037902,
 'renta bruta': 29.53,
 'renta neta': 23.976}

In [369]:
def get_(nombre, dic):
    return dic

In [446]:
rentacodigo

,Unnamed: 0,codigo postal,renta bruta media,renta disponible media
0,0,2001,32.036,25.823
1,1,2002,30.426,24.615
2,2,2003,26.866,22.145
3,3,2004,23.072,19.449
4,4,2005,25.098,20.822
...,...,...,...,...
593,593,50190,29.913,24.229
594,594,50194,27.976,22.941
595,595,50620,22.356,18.812
596,596,50720,26.286,21.398


In [437]:
coords[0]['address_components']

[{'long_name': '28018', 'short_name': '28018', 'types': ['postal_code']},
 {'long_name': 'Madrid',
  'short_name': 'Madrid',
  'types': ['locality', 'political']},
 {'long_name': 'Madrid',
  'short_name': 'M',
  'types': ['administrative_area_level_2', 'political']},
 {'long_name': 'Community of Madrid',
  'short_name': 'MD',
  'types': ['administrative_area_level_1', 'political']},
 {'long_name': 'Spain', 'short_name': 'ES', 'types': ['country', 'political']}]

In [443]:
coords2[0]['address_components'][6].get ("long_name")

'28004'

In [444]:
coords2[0]['address_components'][6]["long_name"]

'28004'